## Initializations, module imports, and compatibility headers

In [ ]:
# main python2 compatibility import
from __future__ import print_function, division

# sage compatibility
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt
#import seaborn as sns

#sns.set(style="whitegrid", font_scale=1.7)

## Integrators

The integrators are provided in the separate modules referred below.
The source code can be found in the `integrators` folder

In [ ]:
# RungeKutta4 for the damped harmonic oscillator
# with dumping paramter a and regularized relative
# error function
from integrators.common import rk4, relerr

# Symplectic integrators for the damped harmonic
# oscillator with dumping parameter a
from integrators.oscillator import euler, leapfrog, \
    ruth3 #, leapfrog2, pseudoleapfrog

# Contact integrators as described in the paper
# for the damped harmonic oscillator with dumping parameter a
from integrators.oscillator import contact, symcontact, midpoint

In [ ]:
from integrators.contact import dEL

#def contact(init, tspan, a, timestep): 
#    integrate = dEL(lambda x0,x1,z0,z1,h: .5*((x1-x0)/h)**2 - .25*x0**2 - .25*x1**2 - a*z0)
#    return integrate(init,tspan,timestep)

#def symcontact(init, tspan, a, timestep): 
#    integrate = dEL(lambda x0,x1,z0,z1,h: .5*((x1-x0)/h)**2 - .25*x0**2 - .25*x1**2 - .5*a*z0 - .5*a*z1)
#    return integrate(init,tspan,timestep)

def contact4(init, tspan, a, timestep): 
    clag = lambda x,v,z: .5*v**2 - .5*x**2 - a*z
    def lag(x0,x1,z0,z1,h):
        vmid= (x1-x0)/h
        xmid = .5*(x0+x1) - h**2/8 * (-.5*x0 - .5*x1 - a*vmid) 
        zmid = .5*(z0+z1) - h**2/8 * clag(.5*(x0+x1),vmid,.5*(z0+z1))
        v0 = (-3*x0 + 4*xmid - x1)/h
        v1 = (3*x1 - 4*xmid + x0)/h
        return 1/6*clag(x0,v0,z0) + 2/3*clag(xmid,vmid,zmid) + 1/6*clag(x1,v1,z1)
    integrate = dEL(lag, implicit=False)
    return integrate(init,tspan,timestep)

## Comparative plots generator

In [ ]:
def cmp_plot(init, tspan, a, h, delta, save=False):
    
    # The reference solution is computed using the leapfrog
    # algorithm with much finer error step
    r = rk4(init, tspan, a, h/delta)

    s1 = contact(init, tspan, a, h)
    s2 = symcontact(init, tspan, a, h)
    s3 = contact4(init, tspan, a, h)
    s4 = leapfrog(init, tspan, a, h)
    #s5 = leapfrog2(init, tspan, a, h)
    s6 = ruth3(init, tspan, a, h)
    s7 = rk4(init,tspan,a,h)
    
    timerange = np.arange(tspan[0], tspan[1], h)
    ref = r[::delta,1]
    
    plt.figure(figsize=(20,10))
    
    plt.subplot(211)
    plt.title("Solution for $h={}$, $\\alpha={}$, $(p_0,q_0)={}$".
              format(h, a, tuple(init))
             )
    #plt.plot(timerange, s1[:,1], label="Contact (1st)", linestyle="--")
    #plt.plot(timerange, s2[:,1], label="Contact (2nd)", linestyle="-.")
    plt.plot(timerange, s3[:,1], label="Contact (4th)", linestyle="--")
    #plt.plot(timerange, s4[:,1], label="Leapfrog", linestyle=":")
    #plt.plot(timerange, s5[:,1], label="Leapfrog (single step)", linestyle=":")
    #plt.plot(timerange, s6[:,1], label="Ruth3", linestyle=":")
    plt.plot(timerange, s7[:,1], label="RK4", linestyle="-")
    plt.plot(np.arange(tspan[0], tspan[1], h/delta), r[:,1],
             label="Reference", linestyle="-")
    plt.legend()
    
    plt.subplot(212)
    plt.title("Relative Error")
    #plt.plot(timerange, relerr(ref,s1[:,1]), label="Contact (1st)", linestyle="--")
    #plt.plot(timerange, relerr(ref,s2[:,1]), label="Contact (2nd)", linestyle="-.")
    plt.plot(timerange, relerr(ref,s3[:,1]), label="Contact (4th)", linestyle="--")
    #plt.plot(timerange, relerr(ref,s4[:,1]), label="Leapfrog", linestyle=":")
    #plt.plot(timerange, relerr(ref,s5[:,1]), label="Leapfrog (single step)", linestyle=":")
    #plt.plot(timerange, relerr(ref,s6[:,1]), label="Ruth3", linestyle=":")
    plt.plot(timerange, relerr(ref,s7[:,1]), label="RK4", linestyle="-")
    plt.legend()

    if save:
        name = "damped{}.pdf".format(a)
        plt.savefig(name, format="pdf", transparent=True)
    
    plt.show()

## Comparative Plots

In [ ]:
init = (.5, .5)
tspan = (0.0, 10.0)
a = 0.1
h = 0.1
delta = 10

cmp_plot(init, tspan, a, h, delta)

In [ ]:
#init = np.random.rand(2)
init = (.75,-.25)
delta = 10
tspan = (0.0, 50.0)

for a in [0,.01, 0.1, 1.0, 2.0]:
    if a > .5:
        tspan = (0.0, 15.0)
    for h in [0.1]:
        cmp_plot(init, tspan, a, h, delta) #, save=True)